In [1]:
import pyspark

In [2]:
import pandas as pd 

In [3]:
pd.read_csv("test.csv")

,Name,Age,Experience
0,Nisse,15,7
1,Ann,47,4


In [4]:
from pyspark.sql import SparkSession

In [5]:
spark=SparkSession.builder.appName("test").getOrCreate()

In [6]:
spark

In [7]:
df_pyspark = spark.read.csv("test.csv", header=True, inferSchema=True)
type(df_pyspark)

pyspark.sql.dataframe.DataFrame

In [8]:
df_pyspark.show()

+-----+---+----------+
| Name|Age|Experience|
+-----+---+----------+
|Nisse| 15|         7|
|  Ann| 47|         4|
+-----+---+----------+



In [46]:
df_pyspark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)



In [38]:
df_pyspark.select(["Name", "Experience"]).show()

+-----+----------+
| Name|Experience|
+-----+----------+
|Nisse|         7|
|  Ann|         4|
+-----+----------+



In [47]:
df_pyspark.dtypes

[('Name', 'string'), ('Age', 'int'), ('Experience', 'int')]

In [40]:
df_pyspark.describe().show()

+-------+-----+------------------+------------------+
|summary| Name|               Age|        Experience|
+-------+-----+------------------+------------------+
|  count|    2|                 2|                 2|
|   mean| null|              31.0|               5.5|
| stddev| null|22.627416997969522|2.1213203435596424|
|    min|  Ann|                15|                 4|
|    max|Nisse|                47|                 7|
+-------+-----+------------------+------------------+



In [49]:
df_pyspark = df_pyspark.withColumn("Experience after 2 years", df_pyspark["Experience"]+2)
df_pyspark.show()

+-----+---+----------+------------------------+
| Name|Age|Experience|Experience after 2 years|
+-----+---+----------+------------------------+
|Nisse| 15|         7|                       9|
|  Ann| 47|         4|                       6|
+-----+---+----------+------------------------+



In [61]:
df_pyspark=df_pyspark.drop("Experience after 2 years")
df_pyspark.show()

+-----+---+----------+
| Name|Age|Experience|
+-----+---+----------+
|Nisse| 15|         7|
|  Ann| 47|         4|
+-----+---+----------+



In [63]:
df_pyspark.withColumnRenamed("Name", "New Name").show()

+--------+---+----------+
|New Name|Age|Experience|
+--------+---+----------+
|   Nisse| 15|         7|
|     Ann| 47|         4|
+--------+---+----------+



In [65]:
spark=SparkSession.builder.appName("practise").getOrCreate()

df_pyspark = spark.read.csv("test2.csv", header=True, inferSchema=True)

In [67]:
df_pyspark.show()

+-----+----+----------+------+
| Name| Age|Experience|Salery|
+-----+----+----------+------+
|Nisse|  15|         7|    33|
|  Ann|  47|         4|    42|
|Bengt|null|      null|  null|
| null|  22|        25|  null|
+-----+----+----------+------+



In [68]:
df_pyspark.drop("Name").show()

+----+----------+------+
| Age|Experience|Salery|
+----+----------+------+
|  15|         7|    33|
|  47|         4|    42|
|null|      null|  null|
|  22|        25|  null|
+----+----------+------+



In [71]:
df_pyspark.na.drop(how="any").show()

+-----+---+----------+------+
| Name|Age|Experience|Salery|
+-----+---+----------+------+
|Nisse| 15|         7|    33|
|  Ann| 47|         4|    42|
+-----+---+----------+------+



In [72]:
df_pyspark.na.drop(how="any", thresh=2).show()

+-----+---+----------+------+
| Name|Age|Experience|Salery|
+-----+---+----------+------+
|Nisse| 15|         7|    33|
|  Ann| 47|         4|    42|
| null| 22|        25|  null|
+-----+---+----------+------+



In [73]:
df_pyspark.na.drop(how="any", thresh=3).show()

+-----+---+----------+------+
| Name|Age|Experience|Salery|
+-----+---+----------+------+
|Nisse| 15|         7|    33|
|  Ann| 47|         4|    42|
+-----+---+----------+------+



In [74]:
df_pyspark.na.drop(how="any", subset=["Age"]).show()

+-----+---+----------+------+
| Name|Age|Experience|Salery|
+-----+---+----------+------+
|Nisse| 15|         7|    33|
|  Ann| 47|         4|    42|
| null| 22|        25|  null|
+-----+---+----------+------+



In [83]:

df_pyspark.na.fill(value="Missing",subset="Name").show()

+-------+----+----------+------+
|   Name| Age|Experience|Salery|
+-------+----+----------+------+
|  Nisse|  15|         7|    33|
|    Ann|  47|         4|    42|
|  Bengt|null|      null|  null|
|Missing|  22|        25|  null|
+-------+----+----------+------+



In [85]:
df_pyspark.na.fill(value=0,subset=["Age", "Experience"]).show()

+-----+---+----------+------+
| Name|Age|Experience|Salery|
+-----+---+----------+------+
|Nisse| 15|         7|    33|
|  Ann| 47|         4|    42|
|Bengt|  0|         0|  null|
| null| 22|        25|  null|
+-----+---+----------+------+



In [89]:
from pyspark.ml.feature import Imputer

imputer = Imputer(inputCols=["Age", "Experience"], outputCols=["Age_filled", "Experience_filled"]).setStrategy("mean")

In [90]:
imputer.fit(df_pyspark).transform(df_pyspark).show()

+-----+----+----------+------+----------+-----------------+
| Name| Age|Experience|Salery|Age_filled|Experience_filled|
+-----+----+----------+------+----------+-----------------+
|Nisse|  15|         7|    33|        15|                7|
|  Ann|  47|         4|    42|        47|                4|
|Bengt|null|      null|  null|        28|               12|
| null|  22|        25|  null|        22|               25|
+-----+----+----------+------+----------+-----------------+



In [107]:
df_pyspark.filter((df_pyspark["Age"]<47) & (df_pyspark["Experience"]>10)).show()


+----+---+----------+------+
|Name|Age|Experience|Salery|
+----+---+----------+------+
|null| 22|        25|  null|
+----+---+----------+------+



In [108]:
df_pyspark.filter(df_pyspark["Age"]<47).select("Name").show()

+-----+
| Name|
+-----+
|Nisse|
| null|
+-----+



In [110]:
df_pyspark.filter(~(df_pyspark["Age"]<47)).select("Name").show()

+----+
|Name|
+----+
| Ann|
+----+



In [117]:
df_pyspark = spark.read.csv("test3.csv", header=True, inferSchema=True)
df_pyspark.show()

+-----+----------+------+
| Name|Department|Salery|
+-----+----------+------+
|Nisse|      dep1|  71.0|
|  Ann|      dep1|  41.0|
|Bengt|      dep2|  55.0|
|Sunny|      dep2|  22.0|
|Sunny|      dep3|  11.0|
|Bengt|      dep3|   2.0|
+-----+----------+------+



In [116]:

df_pyspark.groupBy("Name").sum().show()

+-----+-----------+
| Name|sum(Salery)|
+-----+-----------+
|Bengt|       57.0|
|Sunny|       33.0|
|  Ann|       41.0|
|Nisse|       71.0|
+-----+-----------+



In [118]:
df_pyspark.groupBy("Department").sum().show()

+----------+-----------+
|Department|sum(Salery)|
+----------+-----------+
|      dep3|       13.0|
|      dep2|       77.0|
|      dep1|      112.0|
+----------+-----------+



In [119]:
df_pyspark.groupBy("Department").mean().show()

+----------+-----------+
|Department|avg(Salery)|
+----------+-----------+
|      dep3|        6.5|
|      dep2|       38.5|
|      dep1|       56.0|
+----------+-----------+



In [120]:
df_pyspark.groupBy("Department").count().show()

+----------+-----+
|Department|count|
+----------+-----+
|      dep3|    2|
|      dep2|    2|
|      dep1|    2|
+----------+-----+



In [121]:
df_pyspark.groupBy("Name").count().show()

+-----+-----+
| Name|count|
+-----+-----+
|Bengt|    2|
|Sunny|    2|
|  Ann|    1|
|Nisse|    1|
+-----+-----+



In [122]:
df_pyspark.agg({"Salery": "mean"}).show()

+------------------+
|       avg(Salery)|
+------------------+
|33.666666666666664|
+------------------+



In [123]:
df_pyspark.groupBy("Name").max().show()

+-----+-----------+
| Name|max(Salery)|
+-----+-----------+
|Bengt|       55.0|
|Sunny|       22.0|
|  Ann|       41.0|
|Nisse|       71.0|
+-----+-----------+



In [124]:
df_pyspark.groupBy("Name").avg().show()

+-----+-----------+
| Name|avg(Salery)|
+-----+-----------+
|Bengt|       28.5|
|Sunny|       16.5|
|  Ann|       41.0|
|Nisse|       71.0|
+-----+-----------+



In [27]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

spark = SparkSession.builder.appName("TestML").getOrCreate()

data = spark.read.csv("test4.csv", header=True, inferSchema=True)

#data.printSchema()
#data.columns


featureassembler = VectorAssembler(inputCols = ["Age", "Experience"], outputCol="Independet features")
output=featureassembler.transform(data)

#output.show()
#output.columns

finalized_data = output.select("Independet features", "Salery")

#finalized_data.show()

train_data, validate_data = finalized_data.randomSplit([0.6, 0.4])
regressor = LinearRegression(featuresCol="Independet features", labelCol="Salery")
regressor = regressor.fit(train_data)

regressor.coefficients
regressor.intercept

pred_result = regressor.evaluate(validate_data)


pred_result.predictions.show()
pred_result.meanAbsoluteError, pred_result.meanSquaredError
pred_result.r2

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salery: integer (nullable = true)

+-----+---+----------+------+-------------------+
| Name|Age|Experience|Salery|Independet features|
+-----+---+----------+------+-------------------+
|Nisse| 15|         7|    33|         [15.0,7.0]|
|  Ann| 47|         4|    42|         [47.0,4.0]|
|Bengt| 33|        44|    55|        [33.0,44.0]|
|Julia| 22|        25|    44|        [22.0,25.0]|
+-----+---+----------+------+-------------------+

+-------------------+------+
|Independet features|Salery|
+-------------------+------+
|         [15.0,7.0]|    33|
|         [47.0,4.0]|    42|
|        [33.0,44.0]|    55|
|        [22.0,25.0]|    44|
+-------------------+------+

+-------------------+------+------------------+
|Independet features|Salery|        prediction|
+-------------------+------+------------------+
|        [22.0,25.0]|    44|  38.9219330855019|
|        [33.0

-0.6887680124481199